In [2]:
!git clone https://github.com/jcnrn/usptoSearch.git
!pip install autocorrect
!pip3 install fuzzywuzzy[speedup]
!pip install uspto-opendata-python

fatal: destination path 'usptoSearch' already exists and is not an empty directory.


In [3]:
import os
import nltk.data
import numpy
import pandas as pd
from os import listdir
from os.path import isfile, join
from nltk.tokenize import TreebankWordTokenizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
treebank_tokenizer = TreebankWordTokenizer()
import itertools
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import re
import math
from collections import Counter


from nltk.corpus import stopwords
stop = stopwords.words('english')

from autocorrect import spell 

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from nltk.stem import SnowballStemmer

import requests
from pandas.io.json import json_normalize
import json

os.chdir('/content/usptoSearch')
os.getcwd()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'/content/usptoSearch'

In [4]:
#import data
df = pd.read_csv('usptoSubsetData.csv')
print(len(df))
df.head()

10462


,pat_no,claim_no,claim_txt,dependencies,ind_flg,appl_id
0,4039651,2,"2. The process of claim 1, wherein in process ...",1,0,NaN
1,4039651,1,1. In the process for the production of hydrog...,NaN,1,NaN
2,4039652,23,23. A method as in claim 1 wherein said substa...,1,0,5405316.0
3,4039652,29,29. A method as in claim 26 wherein said immob...,26,0,5405316.0
4,4039652,24,24. A method as in claim 23 wherein said subst...,23,0,5405316.0


In [5]:
df1=df.copy()
#concatenate claims by patent number
df2 = df1.sort_values(['pat_no', 'claim_no']).groupby('pat_no', sort=False).claim_txt.apply(' '.join).reset_index(name='concat_claims')
df2.to_csv('concatenated.csv')
print(len(df))
df2.head()


10462


,pat_no,concat_claims
0,4039651,1. In the process for the production of hydrog...
1,4039652,1. A method for the quantitative determination...
2,4039653,1. A tablet for releasing a relatively uniform...
3,4039654,1. A prostanoic acid derivative of the formula...
4,4039655,1. A composition of matter useful in caries pr...


In [6]:
df3 = df2.copy()
df3['processed'] = df3['concat_claims']

#preprocess data
stemmer = SnowballStemmer('english')   #snowball stemmer chosen over porter due for its aggressiveness

#preprocess data
df3['processed'] = df3['processed'].str.replace('\d+', '') #remove digits
df3['processed'] = df3['processed'].str.replace('[^\w\s]','') #remove punctuation
df3['processed'] = df3['processed'].str.lower() #lowercase
#df3['processed'] = df3['processed'].apply(lambda x: " ".join([spell(i) for i in x.split()]))   #spellcheck             #computationally too intensive
df3['processed']= df3['processed'].apply(treebank_tokenizer.tokenize) #tokenize
df3['processed'] = df3['processed'].apply(lambda x: [y for y in x if y not in stop]) #remove stop words
#df3['processed'] = df3['processed'].apply(lambda x: [wordnet_lemmatizer.lemmatize(item) for item in x]) #lemmatize    #stemming chosen over lemmatize
df3['processed'] = df3['processed'].apply(lambda x: [stemmer.stem(item) for item in x]) #stem
df3.head()


,pat_no,concat_claims,processed
0,4039651,1. In the process for the production of hydrog...,"[process, product, hydrogen, oxygen, water, me..."
1,4039652,1. A method for the quantitative determination...,"[method, quantit, determin, specif, bind, subs..."
2,4039653,1. A tablet for releasing a relatively uniform...,"[tablet, releas, relat, uniform, quantiti, med..."
3,4039654,1. A prostanoic acid derivative of the formula...,"[prostano, acid, deriv, formula, str, wherein,..."
4,4039655,1. A composition of matter useful in caries pr...,"[composit, matter, use, cari, prevent, compris..."


In [7]:
#create keyword list of 40 of the most commonly used words
df4=df3.copy()
df4['keywords'] = df4['processed'].apply(lambda x: [k for k, v in Counter(x).most_common(30)])
dfIndex = df4
dfIndex.to_csv('patentIndex.csv')
dfIndex = df4.drop(['concat_claims'], axis=1)
dfIndex.head()

,pat_no,processed,keywords
0,4039651,"[process, product, hydrogen, oxygen, water, me...","[iron, oxid, chlorid, step, ii, process, hydro..."
1,4039652,"[method, quantit, determin, specif, bind, subs...","[said, matrix, sampl, wherein, method, predete..."
2,4039653,"[tablet, releas, relat, uniform, quantiti, med...","[tablet, said, materi, medica, releas, odormas..."
3,4039654,"[prostano, acid, deriv, formula, str, wherein,...","[rsup, acid, prostano, deriv, hydrogen, claim,..."
4,4039655,"[composit, matter, use, cari, prevent, compris...","[group, compound, said, composit, oral, german..."


In [8]:
#rawSearch=(input('Enter your search:'))
#aggressive spellchecker/autocorrect simplifies search
#but has obvious limitations 
rawSearch = 'liquid1 crystalss displaying'


def searchPreprocessing(rawSearch):
  stemmer = SnowballStemmer('english')
  
  searchQuery = rawSearch
  searchQuery = searchQuery.lower()
  searchQuery = ' '.join(map(spell, searchQuery.split()))
  searchQuery = searchQuery.replace('[^\w\s]','')
  searchQuery = searchQuery.replace('\d+', '')
  searchQuery = searchQuery.lower()
  searchQuery = treebank_tokenizer.tokenize(searchQuery)
  searchQuery = [word for word in searchQuery if word not in stopwords.words('english')]
  searchQuery = [stemmer.stem(word) for word in searchQuery]
  
  print("Searching '" + str(rawSearch) + "' as: " + str(searchQuery))
  return searchQuery
  

searchQuery = searchPreprocessing(rawSearch)



Searching 'liquid1 crystalss displaying' as: ['liquid', 'crystal', 'display']


In [0]:
#define cosine similarity function
def cosine_similarity(document_1_data, document_2_data):
    def stringify(whatever):
        newString = " ".join(whatever)
        return newString
    
    
    WORD = re.compile(r'\w+') 
    
    def get_cosine(vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        
        if not denominator:
            return 0.0
        else:
            return float(numerator) / denominator
        
    def text_to_vector(text):
        words = WORD.findall(text)
        return Counter(words)
    
    text1 = stringify(document_1_data)
    text2 = stringify(document_2_data)
    
    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)
    
    cosine = get_cosine(vector1, vector2)
    
    return(cosine)

    
#test   
#doc1=searchQuery
#doc2=dfIndex.iloc[0,1]
#cosine_similarity(doc1, doc2)

In [0]:
#find cosine values and rank them
searchRank = []
doc1=searchQuery

for index, row in dfIndex.iterrows():
  doc2 = dfIndex.iloc[index,1]
  procesCos = (cosine_similarity(doc1,doc2))
  doc2 = dfIndex.iloc[index,2]
  keyworCos = (cosine_similarity(doc1,doc2))
  temp = (dfIndex.iloc[index,0], procesCos, keyworCos)
  searchRank.append(temp)


searchRank=pd.DataFrame(searchRank ,columns=['patNo', 'processedCosRank', 'keywordCosRank'])


#rank by processed tokens
searchRank = searchRank.sort_values(by=['processedCosRank'], ascending=False)
#rank by keyword matches
#searchRank = searchRank.sort_values(by=['keywordCosRank'], ascending=False)
searchRank.head(10)

,patNo,processedCosRank,keywordCosRank
915,4040567,0.305293,0.105409
152,4039803,0.286308,0.210819
75,4039726,0.266242,0.105409
1067,4040719,0.245722,0.316228
1073,4040725,0.216401,0.105409
542,4040193,0.190189,0.105409
194,4039845,0.181733,0.105409
1068,4040720,0.181273,0.316228
230,4039881,0.176734,0.105409
178,4039829,0.165418,0.105409


In [0]:
#take top ranking patent number
patNo = str(searchRank.iloc[0,0])

url = 'http://www.patentsview.org/api/patents/query?q={"patent_number":"'+ patNo + '"}&f=["inventor_first_name","inventor_last_name","patent_number", "patent_title", "assignee_country","assignee_organization"]'

r = requests.get(url)
json_data = r.json()

#for r in json_data['patents']:
#    print(r)

print(json.dumps(json_data, sort_keys=True, indent=0))

{
"count": 1,
"patents": [
{
"assignees": [
{
"assignee_country": "US",
"assignee_key_id": "127555",
"assignee_organization": "NuPro, Inc."
}
],
"inventors": [
{
"inventor_first_name": "Charles D.",
"inventor_key_id": "16821",
"inventor_last_name": "McNeely"
}
],
"patent_number": "4040567",
"patent_title": "Heating unit"
}
],
"total_patent_count": 1
}


In [0]:
#keywords = dfIndex['keywords']
#keywords.head()

#fuzzywuzzy keyword matching not great
#process.extract(str(searchQuery), keywords, limit=4, scorer=fuzz.token_sort_ratio)

In [0]:
#Doesn't work on many patents. Using PatentView API Instead
#from uspto.peds.client import UsptoPatentExaminationDataSystemClient
#client = UsptoPatentExaminationDataSystemClient()

#result = client.download_document('3987485')